## Data

### Done

1. load using pandas

## State Representation

### Done

1. Transform categorical idices

### TODO

1. features bucketised
2. weights (real values floating point numbers, normalised (?))
3. normalise cost?


In [215]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [181]:
all_data = pd.read_csv('./src/data/insurance.csv')
train, test = train_test_split(all_data)
train_x_raw = pd.DataFrame(train, columns=train.columns.drop('charges'))
train_y_raw = pd.DataFrame(train.charges)
test_x_raw = pd.DataFrame(test, columns=test.columns.drop('charges'))
test_y_raw = pd.DataFrame(test.charges)

In [182]:
print(train_x_raw.head())
print(train_y_raw.head())
print(test_x_raw.head())
print(test_y_raw.head())

      age     sex    bmi  children smoker     region
144    30    male  28.69         3    yes  northwest
538    46  female  28.05         1     no  southeast
121    18    male  23.75         0     no  northeast
396    43  female  34.40         3     no  southwest
1107   50  female  26.22         2     no  northwest
         charges
144   20745.9891
538    8233.0975
121    1705.6245
396    8522.0030
1107  10493.9458
     age     sex     bmi  children smoker     region
815   20  female  31.460         0     no  southeast
6     46  female  33.440         1     no  southeast
568   49  female  31.900         5     no  southwest
281   54    male  40.565         3    yes  northeast
619   55  female  37.100         0     no  southwest
         charges
815   1877.92940
6     8240.58960
568  11552.90400
281  48549.17835
619  10713.64400


In [211]:
# we use this to map the categorical features into indices, we can map back later for presentation
def extract_category_mappings(dataframe, category_columns):
    to_map = lambda uniques: {v:k[0] for k, v in np.ndenumerate(uniques)}
    return [(to_map(dataframe[col].unique()), col) for col in category_columns]

In [212]:
mappings = extract_category_mappings(all_data, ['sex', 'smoker', 'region'])

In [213]:
# transform the categorical features to binary representations of their indices;
# we might want to bucketize the continuous features?
def transform_features(dataframe, mappings):
    transformed_df = pd.DataFrame(dataframe, columns=['age', 'bmi', 'children'])
    
    for mapping, name in mappings:
        for i in mapping.values():
            transformed_df[name + str(i)] = dataframe[name].apply(lambda x: int(mapping[x] == i))

    return transformed_df

train_x = transform_features(train_x_raw, mappings)
test_x = transform_features(test_x_raw, mappings)

In [214]:
train_x.head()

,age,bmi,children,sex0,sex1,smoker0,smoker1,region0,region1,region2,region3
144,30,28.69,3,0,1,1,0,0,0,1,0
538,46,28.05,1,1,0,0,1,0,1,0,0
121,18,23.75,0,0,1,0,1,0,0,0,1
396,43,34.40,3,1,0,0,1,1,0,0,0
1107,50,26.22,2,1,0,0,1,0,0,1,0


In [222]:
# use scikitlearn minmaxscaler, then on the way out use inverse_transform to get the real values again
# TODO: fix this so it works on a column/series, and doesn't reset indices
def normalize(series):
    scaler = preprocessing.MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(series))

normalize(train_y_raw)

,0
0,0.313136
1,0.113373
2,0.009166
3,0.117986
4,0.149467
5,0.188102
6,0.026657
7,0.022396
8,0.034298
9,0.158912


In [ ]:
def predict(features, weights):
    return np.dot(features, weights)